### 데이터 정합성 점검: CSV vs 이미지 폴더 (id 기준)

In [16]:
import os
import pandas as pd

csv_path = "../../csv/data_0826.csv" # CSV 경로
img_root = "../../data/preprocessed/images" # 이미지 루트

# 1) CSV 필터링: model, condition 둘 다 존재(결측/빈문자열 아님)
df = pd.read_csv(csv_path, index_col=0)
df["model"] = df["model"].astype(str).str.strip()
df["condition"] = df["condition"].astype(str).str.strip()
valid = df.dropna(subset=["model", "condition"])
valid = valid[(valid["model"] != "") & (valid["condition"] != "")]

# 중복 ID가 있으면 하나의 ID가 여러 행일 수 있으므로 unique로 비교
csv_ids = set(valid["id"].astype(str).unique())

# 2) 폴더 ID 집합
folder_ids = {
    d for d in os.listdir(img_root)
    if os.path.isdir(os.path.join(img_root, d))
}

# 3) 요약
print("CSV(유효) ID 수:", len(csv_ids))
print("이미지 폴더 수 :", len(folder_ids))
print("동일 여부 :", len(csv_ids) == len(folder_ids))

csv_only = csv_ids - folder_ids   # CSV엔 있는데 폴더 없음
fold_only = folder_ids - csv_ids  # 폴더만 있고 CSV(유효행)엔 없음

print("CSV에만 있는 ID 수 :", len(csv_only))
print("폴더만 있는 ID 수  :", len(fold_only))

# 필요하면 목록 파일로 저장
pd.Series(sorted(csv_only)).to_csv("missing_folders_from_csv_ids.csv", index=False, header=["id"])
pd.Series(sorted(fold_only)).to_csv("orphan_folders_without_csv_rows.csv", index=False, header=["id"])

# (선택) 유효 CSV에서 중복 ID 체크
dup = valid["id"].duplicated().sum()
print("유효 CSV 내 중복 ID 개수:", dup)


CSV(유효) ID 수: 5127
이미지 폴더 수 : 5127
동일 여부 : True
CSV에만 있는 ID 수 : 0
폴더만 있는 ID 수  : 0
유효 CSV 내 중복 ID 개수: 0


### 스토케 유모차 데이터셋의 이미지 1장씩만 가져오기

In [46]:
from pathlib import Path
import shutil

def get_first_images(img_directory, out_directory):
    img_root = Path(img_directory) # 이미지 루트 폴더
    out_dir  = Path(out_directory)

    out_dir.mkdir(parents=True, exist_ok=True)

    exts = ('.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tif', '.tiff')

    first_images = []  # (폴더명, 첫번째 이미지 전체경로)

    copied = 0
    for folder in img_root.iterdir():
        if not folder.is_dir():
            continue  # 폴더만 대상

        # 정렬 없이, 이터레이터에서 '첫 매칭 하나'만 꺼냄
        first_img = next(
            (p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts),
            None
        ) # 없으면 None, 있으면 첫 매칭 Path
        if first_img is None:
            continue

        # 저장 파일명: 폴더명(uuid) + 원래 확장자
        dst = out_dir / f"{folder.name}{first_img.suffix.lower()}"

        shutil.copy2(first_img, dst)

        copied += 1

    print("복사된 이미지 수:", copied)


In [47]:
get_first_images("../../data/preprocessed/images", "../../data/features/first_images")

복사된 이미지 수: 5127


### 각이미지들에 image_path행을 만들어주기

In [17]:
# df에 'image_path' 컬럼이 없으면 새로 만들기
if "image_path" not in df.columns:
    df["image_path"] = "data/features/first_images/" + df["id"].astype(str) + ".jpg"

In [12]:
df['image_path'].isna().sum()

np.int64(0)

In [13]:
df.head()

,id,title,detail,condition,is_completed,price,location,source,model,model_type,image_path
0,00035af0-d486-4be9-8be2-b1570b31ee81,스토케 유모차,"사용감은 있으나 작동하는데 문제는 없습니다! 발받침, 컵홀더 있어요. 지금 딱 겨울...",사용감 적음,True,100000.0,서울특별시,daangn,nan,NaN,data/features/first_images/00035af0-d486-4be9-...
1,001bf36c-66f6-4fc9-9bf8-de21ae6819d8,스토케 익스플로리 유모차,스토케 유모차 22년9월 신세계강남점에서 구매첫째둘째 다타고 내놓습니다 정품인증완료...,사용감 적음,False,527790.0,주안6동,bungae,explori,NaN,data/features/first_images/001bf36c-66f6-4fc9-...
2,001e0955-8abc-4b06-ba0a-1530e5d7891c,스토케유모차,"삼성 벽걸이 에어컨 AR06D1150HZ300,000원삼양동·2일 전",nan,True,65000.0,NaN,daangn,nan,NaN,data/features/first_images/001e0955-8abc-4b06-...
3,00305b3c-f670-4430-bdbb-c154516a2098,스토케유모차 익스플로리엑스(구성품 포함),스토케익스플로리엑스 유모차 판매합니다 구성품에 *유모차라이너(듀라론 양면 사계절용)...,새 상품,True,70000.0,부산광역시,daangn,explori,NaN,data/features/first_images/00305b3c-f670-4430-...
4,004ad797-ed9e-4acb-a51a-eab9b3fdc456,스토케 유모차,스토케 유모차입니다. 사용감은 있지만 깨끗한 편입니다. 구성품 왠만한건 다 있습니다...,사용감 적음,True,50000.0,경기도,daangn,nan,NaN,data/features/first_images/004ad797-ed9e-4acb-...


### location, model, condition, model_type의 결측치값들을 "unknown" 으로 처리

In [32]:
# nan값은 학습에서 자동으로 무시됨으로 범주형 결측치도 학습 신호가 되니까 버리지 않고 "unknown"으로 치환
for col in ["location","model","condition","model_type"]:
    if col not in df.columns:
        df[col] = "unknown"
    else:
        df[col] = df[col].fillna("unknown").astype(str)

### CSV 파일 저장

In [ ]:
df.to_csv("../../csv/cleaned_total.csv", index=False)

In [7]:
import pandas as pd

df = pd.read_csv("../../csv/cleaned_total.csv")

In [5]:
df.head()

,id,title,detail,condition,is_completed,price,location,source,model,model_type,image_path
0,00035af0-d486-4be9-8be2-b1570b31ee81,스토케 유모차,"사용감은 있으나 작동하는데 문제는 없습니다! 발받침, 컵홀더 있어요. 지금 딱 겨울...",사용감 적음,True,100000.0,서울특별시,daangn,unknown,unknown,data/features/first_images/00035af0-d486-4be9-...
1,001bf36c-66f6-4fc9-9bf8-de21ae6819d8,스토케 익스플로리 유모차,스토케 유모차 22년9월 신세계강남점에서 구매첫째둘째 다타고 내놓습니다 정품인증완료...,사용감 적음,False,527790.0,주안6동,bungae,explori,unknown,data/features/first_images/001bf36c-66f6-4fc9-...
2,001e0955-8abc-4b06-ba0a-1530e5d7891c,스토케유모차,"삼성 벽걸이 에어컨 AR06D1150HZ300,000원삼양동·2일 전",NaN,True,65000.0,unknown,daangn,unknown,unknown,data/features/first_images/001e0955-8abc-4b06-...
3,00305b3c-f670-4430-bdbb-c154516a2098,스토케유모차 익스플로리엑스(구성품 포함),스토케익스플로리엑스 유모차 판매합니다 구성품에 *유모차라이너(듀라론 양면 사계절용)...,새 상품,True,70000.0,부산광역시,daangn,explori,unknown,data/features/first_images/00305b3c-f670-4430-...
4,004ad797-ed9e-4acb-a51a-eab9b3fdc456,스토케 유모차,스토케 유모차입니다. 사용감은 있지만 깨끗한 편입니다. 구성품 왠만한건 다 있습니다...,사용감 적음,True,50000.0,경기도,daangn,unknown,unknown,data/features/first_images/004ad797-ed9e-4acb-...


In [25]:
df.isna().sum()

id              0
title           0
detail          1
condition       0
is_completed    0
price           0
location        0
source          0
model           0
model_type      0
image_path      0
dtype: int64

In [33]:
df.head()

,id,title,detail,condition,is_completed,price,location,source,model,model_type,image_path
0,00035af0-d486-4be9-8be2-b1570b31ee81,스토케 유모차,"사용감은 있으나 작동하는데 문제는 없습니다! 발받침, 컵홀더 있어요. 지금 딱 겨울...",사용감 적음,True,100000.0,서울특별시,daangn,unknown,unknown,data/features/first_images/00035af0-d486-4be9-...
1,001bf36c-66f6-4fc9-9bf8-de21ae6819d8,스토케 익스플로리 유모차,스토케 유모차 22년9월 신세계강남점에서 구매첫째둘째 다타고 내놓습니다 정품인증완료...,사용감 적음,False,527790.0,주안6동,bungae,explori,unknown,data/features/first_images/001bf36c-66f6-4fc9-...
2,001e0955-8abc-4b06-ba0a-1530e5d7891c,스토케유모차,"삼성 벽걸이 에어컨 AR06D1150HZ300,000원삼양동·2일 전",unknown,True,65000.0,unknown,daangn,unknown,unknown,data/features/first_images/001e0955-8abc-4b06-...
3,00305b3c-f670-4430-bdbb-c154516a2098,스토케유모차 익스플로리엑스(구성품 포함),스토케익스플로리엑스 유모차 판매합니다 구성품에 *유모차라이너(듀라론 양면 사계절용)...,새 상품,True,70000.0,부산광역시,daangn,explori,unknown,data/features/first_images/00305b3c-f670-4430-...
4,004ad797-ed9e-4acb-a51a-eab9b3fdc456,스토케 유모차,스토케 유모차입니다. 사용감은 있지만 깨끗한 편입니다. 구성품 왠만한건 다 있습니다...,사용감 적음,True,50000.0,경기도,daangn,unknown,unknown,data/features/first_images/004ad797-ed9e-4acb-...


### train/test 분할

In [8]:
from sklearn.model_selection import train_test_split

# id 단위 고유값 추출
ids = df["id"].unique()

# id 단위로 train/val 분할
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=42)

# 분할된 id를 기반으로 DataFrame 분리
train_df = df[df["id"].isin(train_ids)].copy()
val_df = df[df["id"].isin(val_ids)].copy()

### cat2idx 생성 코드

In [9]:
# 범주형 데이터를 딥러닝 모델 사용을 위해 숫자 벡터로 변환
cat_cols = ["location", "model", "condition", "model_type"]

cat2idx, cardinalities = {}, {}

for c in cat_cols:
    # 중복 제거한 unique 값 모으기 (NaN → 'unknown' 치환)
    cats = train_df[c].astype(str).unique().tolist()

    # 매핑 딕셔너리 만들기
    cat2idx[c] = {v: i for i, v in enumerate(cats)}
    cardinalities[c] = len(cats)   # 각 컬럼마다 카테고리 개수
print(cat2idx)
print(cardinalities)

tab_dim = sum(cardinalities[c] for c in cat_cols)  # 또는: sum(cardinalities.values())

{'location': {'서울특별시': 0, '주안6동': 1, 'unknown': 2, '부산광역시': 3, '경기도': 4, '울산광역시': 5, '대전광역시': 6, '제주특별자치도': 7, '인천광역시': 8, '세종특별자치시': 9, '대구광역시': 10, '광주광역시': 11, '원미1동': 12, '수색동': 13, '태평2동': 14, '검암경서동': 15, '잠실2동': 16, '월송동': 17, '반포3동': 18, '길동': 19, '보람동': 20, '상현1동': 21, '성내1동': 22, '봉명동': 23, '잠실본동': 24, '광교2동': 25, '개봉제2동': 26, '용이동': 27, '탄방동': 28, '중계4동': 29, '정관읍': 30, '가정1동': 31, '고덕제1동': 32, '미사1동': 33, '소공동': 34, '치평동': 35, '정자동': 36, '온양6동': 37, '혜화동': 38, '송도5동': 39, '용암1동': 40, '교남동': 41, '송부동': 42, '면목본동': 43, '약수동': 44, '구로제5동': 45, '공덕동': 46, '쌍문제4동': 47}, 'model': {'unknown': 0, 'explori': 1, 'yoyo': 2, 'beat': 3, 'crusi': 4, 'scoot': 5, 'trailz': 6, 'trailz, crusi': 7, 'explori, trailz': 8, 'trailz, beat': 9, 'yoyo, crusi': 10, 'explori, scoot': 11, 'yoyo, explori': 12, 'explori, trailz, crusi': 13, 'yoyo, trailz': 14, 'yoyo, beat': 15, 'explori, crusi': 16, 'explori, beat': 17}, 'condition': {'사용감 적음': 0, 'nan': 1, '새 상품': 2, '사용감 많음': 3}, 'model_type': {'unknow

### 이미지를 EXIF 교정 후, 비율 편차(diff_thresh)로 정사각 크롭/패딩 → 최종 256 리사이즈

In [10]:
from PIL import Image, ImageOps

def custom_crop_and_resize(
    img: Image.Image,
    target_size: int = 256,
    diff_thresh: float = 0.33,       # 1:1에서 dev가 이 값 초과면 크롭(내용 보존↑면 0.33~0.35)
    pad_color=(114,114,114),
    resample=Image.BICUBIC,
):
    """
    - |w/h - 1| > diff_thresh면 중앙 '크롭', 아니면 '패딩'으로 정사각
    - 그 다음에만 target_size로 리사이즈 (비율 왜곡 없음)
    """
    # EXIF 방향 보정 + RGB
    img = ImageOps.exif_transpose(img).convert("RGB") # EXIF Orientation을 실제 픽셀에 반영해 올바른 방향으로 정규화

    width, height = img.size
    r = max(width/height, height/width) # r >= 1
    dev = r - 1.0 # dev >= 0 (정사각이면 0, 1.33:1이면 0.33)

    # 가로 세로 비율 차이가 크면 크롭을 적용합니다.
    if dev > diff_thresh:
        side = min(width, height)
        left = (width - side) // 2
        top  = (height - side) // 2
        img = img.crop((left, top, left + side, top + side))
    # 비율 차이가 작으면 패딩을 적용하여 정사각형으로 만듭니다.
    else:
        max_side = max(width, height)
        new_img = Image.new('RGB', (max_side, max_side), pad_color)
        new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
        img = new_img
    
    return img.resize((target_size, target_size), resample=resample)

### 원핫 인코딩 만들기

In [11]:
import torch.nn.functional as F

def idx_to_onehot(x_tab_idx: torch.Tensor, cardinalities: dict, cat_cols: list):
    pieces = []
    for i, c in enumerate(cat_cols):
        one = F.one_hot(x_tab_idx[:, i], num_classes=cardinalities[c])  # (B, |c|)
        pieces.append(one)
    return torch.cat(pieces, dim=1).float()  # (B, tab_dim)

### 학습 입력 준비: StokkePriceImageDataset

In [12]:
import numpy as np, torch, torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image, ImageFile
# 손상되었거나(끝이 잘린) 이미지를 PIL이 가능하면 읽도록 허용
ImageFile.LOAD_TRUNCATED_IMAGES = True

class StokkePriceImageDataset(Dataset):
    def __init__(self, df, test_type ="train", log_target = True,
                target_size = 256, diff_thresh = 0.33, pad_color = (114,114,114),
                cat_cols=("location", "model", "condition", "model_type"),
                cat2idx=None): # cat_idx : categorical to index(범주형 변수를 숫자 인덱스로 변환)
        self.df = df.reset_index(drop=True)
        self.log_target = log_target
        self.target_size = target_size
        self.diff_thresh = diff_thresh
        self.pad_color = pad_color
        self.cat_cols = list(cat_cols)
        self.cat2idx = cat2idx

        base = [
            T.Lambda(lambda img: custom_crop_and_resize(
                img, target_size=self.target_size, 
                diff_thresh=self.diff_thresh,
                pad_color=self.pad_color
            )),
            T.ToTensor(),# ← 이제 크기 고정(256x256)
            T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # ImageNet 통계 | 정규화
        ]
        if test_type == 'train':
            # ToTensor() 앞(= index 1)에 증강을 끼워 넣기
            base.insert(1, T.RandomHorizontalFlip(0.5))
            # 밝기/대비/채도/색상을 랜덤으로 흔들어 조명·색감 변화
            # 정규화 진행
            base.insert(1, T.ColorJitter(0.15,0.15,0.05,0.02))
            
        self.tf = T.Compose(base)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        r = self.df.iloc[i] # df에서 i번째 행꺼내기
        # 이미지는 결측치가 없음
        img = Image.open(r["image_path"]) # RGB 변환/ EXIF 처리는 커스텀에서 진행
        x_img = self.tf(img)
        
        # 범주형 → 인덱스
        cats_idx = []
        for c in self.cat_cols:
            # 행의 컬럼 c의값을 꺼내는데 그 값이 없으면 'unknown'이 기본값
            v = r.get(c, "unknown")
            # 값이 None 이거나 float인데 NaN 이면 'unknown'으로 치환
            if v is None or (isinstance(v, float) and np.isnan(v)):
                v = "unknown"
            cats_idx.append(self.cat2idx[c].get(str(v), 0))
        # [location_idx <- 0, model_idx <- 2, condition_idx<- 3, model_type_idx<- 2] 같은 리스트를 PyTorch 텐서로 변환
        x_tab = torch.tensor(cats_idx, dtype=torch.long) # <- Embedding 레이어에 들어갈 인덱스는 정수형이어야한다

        # 타깃
        y = float(r["price"])
        if self.log_target:
            y = np.log1p(y)

        # x_tab: [0, 0, 1, 0]
        # x_tab은 나중에 원핫벡터를 직접 넣어줄것
        return x_img, x_tab, torch.tensor([y], dtype=torch.float32) # [1] -> 배치 시 [B,1] (MSELoss 계산위함)


### DataLoader 만들기

In [13]:
"""
pin_memory=True:
    - GPU 학습 시 필수처럼 쓰는 옵션
    - CPU 메모리에서 GPU로 데이터를 보낼 때 더 빠르게 전송
persistent_workers=True:
    - num_workers > 0일 때만 의미 있음
num_workers:
    - 데이터 읽기 병렬화 정도
    - CPU 코어 수에 맞춰 조정. 
"""

from torch.utils.data import DataLoader

# 결측/누락 이미지 경로 제거(권장)
train_df = train_df[train_df["image_path"].notna()].copy()
val_df   = val_df[val_df["image_path"].notna()].copy()

# Dataset (옵션 하이퍼: target_size / diff_thresh)
train_ds = StokkePriceImageDataset(
    train_df, test_type="train", target_size=256, diff_thresh=0.33, 
    log_target=True, cat2idx=cat2idx
)
val_ds = StokkePriceImageDataset(
    val_df,   test_type="val",   target_size=256, diff_thresh=0.33, 
    log_target=True, cat2idx=cat2idx
)

train_loader   = DataLoader(
    train_ds, batch_size=16, shuffle=True, num_workers=4, pin_memory=True,
    persistent_workers=True
)
val_loader   = DataLoader(
    val_ds, batch_size=16, shuffle=False, num_workers=4, pin_memory=True,
    persistent_workers=True
)

### 모델 빌더: ResNet34 / EfficientNet-B3 / ConvNeXt-Tiny / VGG16-BN (회귀 버전)
- 이미지 + (location/model/condition/model_type)을 합치는 멀티모달

In [14]:
import torch.nn as nn
import torchvision.models as tv
import timm

def build_backbone_feature(name: str, pretrained: bool = True):
    """
    멀티 모달(이미지 + 범주형 -> 가격)
    다양한 CNN 백본(ResNet34, EfficientNet-B3, ConvNeXt-Tiny, VGG16-BN)을
    가격 예측(회귀)용으로 초기화해주는 함수.
    - pretrained=True : ImageNet으로 사전학습된 가중치를 불러옴
    - out_dim : 출력 차원은 회귀이므로 1차원으로 설정
    """
    name = name.lower()
    if name == "resnet34":
        # torchvision ResNet34 불러오기 (ImageNet pretrained weights(가중치) 첫번째 버전 사용)
        m = tv.resnet34(weights=tv.ResNet34_Weights.IMAGENET1K_V1 if pretrained else None)
        feat_dim = m.fc.in_features  # 512
        m.fc = nn.Identity() # 마지막 FC(1000 클래스 분류기) 제거
        return m, feat_dim # 제거 후 출력(B, 512) (특징 벡터)
    
    if name == "vgg16_bn":
        m = tv.vgg16_bn(weights=tv.VGG16_BN_Weights.IMAGENET1K_V1 if pretrained else None)
        feat_dim = m.classifier[-1].in_features  # 4096
        m.classifier[-1] = nn.Identity() # 마지막 FC 제거
        return m, feat_dim # 제거 후 출력: (B, 4096)
    
    if name == "efficientnet_b3":
        m = timm.create_model("efficientnet_b3", pretrained=pretrained, num_classes=0)  # feature only
        feat_dim = m.num_features
        return m, feat_dim # 제거후 출력 (B, 1536)
    
    if name == "convnext_tiny":
        m = timm.create_model("convnext_tiny", pretrained=pretrained, num_classes=0)
        feat_dim = m.num_features
        return m, feat_dim # 거 후 출력: (B, 768)
    
    raise ValueError(name)

### 이미지 특징 + 범주형(탭) 특징을 결합해서 가격(1차원) 예측하는 멀티모달 회귀 모델

In [15]:
import torch
import torch.nn as nn
import torchvision.models as tv



class MultiModalLite(nn.Module):
    """
        - backbone_name: 어떤 CNN 백본 쓸지 (예: resnet34, efficientnet 등)
        - tab_dim: 탭 데이터 원-핫 크기
            - 예: location(100종류) + model(50종류) + condition(3종류) + model_type(5종류) → 100+50+3+5 = 158
        - pretrained: 백본을 ImageNet 사전학습 가중치로 초기화할지 여부
        - out_dim: 최종 출력 차원 (가격 → 1)
    """
    def __init__(self, backbone_name: str, tab_dim: int, pretrained=True, out_dim=1):
        super().__init__()
        # 이미지 백본 & feature 차원 (백본이 내놓는 특징 벡터 크기 (ResNet34=512))
        self.backbone, img_feat_dim = build_backbone_feature(backbone_name, pretrained)

        # 최종 예측 레이어: (이미지 feat + 원핫 탭 feature) → 가격(1)
        self.head = nn.Linear(img_feat_dim + tab_dim, out_dim)

    # 순전파 
    def forward(self, x_img, x_tab_onehot): # 탭 원핫 벡터를 받는다
        img_feat = self.backbone(x_img) # 이미지에서 feature vector 추출(B, img_feat_dim)
        if img_feat.ndim == 4: # timm 백본 보호
            img_feat = torch.flatten(torch.mean(img_feat, dim=[2,3]), 1)

        feat = torch.cat([img_feat, x_tab_onehot], dim=1)  # 이미지 feat 하고 탭 feat 이어붙이기: (B, img_feat_dim + tab_dim)
        return self.head(feat) # 최종 예측가격: (B, 1)

### 모델 쓰기

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbones = ["efficientnet_b3", "resnet34", "convnext_tiny", "vgg16_bn"]

models = {
    name: MultiModalLite(
        backbone_name=name,
        tab_dim=tab_dim,
        pretrained=True,
        out_dim=1
    ).to(device)
    for name in backbones
}

models

{'efficientnet_b3': MultiModalLite(
   (backbone): EfficientNet(
     (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (bn1): BatchNormAct2d(
       40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
       (drop): Identity()
       (act): SiLU(inplace=True)
     )
     (blocks): Sequential(
       (0): Sequential(
         (0): DepthwiseSeparableConv(
           (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
           (bn1): BatchNormAct2d(
             40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
             (drop): Identity()
             (act): SiLU(inplace=True)
           )
           (aa): Identity()
           (se): SqueezeExcite(
             (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
             (act1): SiLU(inplace=True)
             (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
             (gate

In [103]:
models['resnet34']

MultiModalLite(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

학습 모델 우선순위
- EfficientNet-B3 → 파라미터 효율 대비 표현력이 좋아서 256에서도 성능 잘 나옴.
- ConvNeXt-Tiny → 최신 구조라 학습 안정적이고 빠름. (둘 다 해보되, 먼저는 B3)

----------------------------------